In [1]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

# Framework for running model on GPU
import torch
import torch.nn as nn
import torch.nn.functional as F

# For Visualising gradient flow across the model
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# For Visualising training loss, accuracy Vs epochs
# from visdom import Visdom

# Basic computation library needed for visdom visualising class
import numpy as np

# Other libraries for data loading, processing etc
import json
import os
import re
import math
import copy
import os
import string
import time

In [0]:
running_device='colab'


In [0]:

def raw_data_loader(running_device):
    """ 
        Function for loading json data into a list
    """
    
    file_dir=''
    file_name='data.json'
    if running_device=='local':
        file_dir='/home/naive/Documents/rohit/Wizard Of Wikipedia/Dataset'
    elif running_device=='colab':
        file_dir='/content/drive/My Drive/Data/Wizard of Wikipedia'
    else:
        print("Invalid running device")
        return
    
    data=os.path.join(file_dir,file_name)
    
    json_data=None
    with open(data) as f:
        json_data=json.load(f)
    
    return json_data

In [6]:
json_data=None

print("Loading Raw Data into a list....")

t1=time.time()
json_data=raw_data_loader(running_device)
t2=time.time()

print("Loading raw data took "+str(t2-t1)+" seconds")


Loading Raw Data into a list....
Loading raw data took 30.799710035324097 seconds


In [0]:
class vocabulary:
    """ Creates vocabulary for our corpus data of conversations"""
    
    def __init__(self):
        """
            self.word2Index(Dictionary): Maps words to tokens
            self.num_words(Scalar)     : Number of distinct words in the vocabulary
            
        """
        
        self.word2Index={'PAD':0,"<START>":1,"<END>":2}
        self.num_words=3
        
    def addSentence(self,sentence):
        """ 
            Adds sentences into vocabulary by splitting them
        """
        
        for word in sentence.split(" "):
            self.addWord(word)
    
    def addWord(self,word):
        
        if word not in self.word2Index.keys():
            self.word2Index[word]=self.num_words
            self.num_words+=1

In [0]:
# Preprocessing functions for the data

def remove_punctuation(sentence):
    # thanks to https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
    return sentence.translate(str.maketrans('', '', string.punctuation)).lower().strip()

In [9]:
def load_data(data_json):
    
    context, knowledge, wizard=[],[],[]
    dict2cnt={}
    count=0
    
    N=len(data_json)
    for id in range(N):
        
        current_knowledge,current_wizard,context_knowledge,previous_context=[],[],[],[]
        tmp_context=[]
        
        current_conv=data_json[id]
        
        topic_chosen=remove_punctuation(current_conv['chosen_topic'])
        tmp_context.append(topic_chosen)
        
        conversation_length=len(current_conv['dialog'])
        
        for i in range(conversation_length):
            
            if current_conv['dialog'][i]['speaker']=='0_Wizard':
                
                dict2cnt[count]=len(current_conv['dialog'][i]['retrieved_passages'])
                count=count+1
                
                for x in range(len(current_conv['dialog'][i]['retrieved_passages'])):
                    
                    idx=current_conv['dialog'][i]['retrieved_passages'][x]
                    
                    for value in idx.values():
                        
                        for j in range(len(value)):
                            value[j]=remove_punctuation(value[j])
                            
                        current_knowledge.append(value)
                
                wizard_dialog=remove_punctuation(current_conv['dialog'][i]['text'])
                
                current_wizard.append(wizard_dialog)
                tmp_context.append(wizard_dialog)
            
            else:
                apperentice_dialog=remove_punctuation(current_conv['dialog'][i]['text'])
                tmp_context.append(apperentice_dialog)
                
        knowledge.append(current_knowledge)
        wizard.append(current_wizard)
        
        final_context=[]
        flag=True
        
        if current_conv['dialog'][0]['speaker']=='0_Wizard':
            flag=False
        
        for i in range(len(tmp_context)):
            temp=[]
            
            for j in range(i+1):
                temp.append(tmp_context[j])
                
            if flag:
                if i%2!=0 and len(final_context)<len(current_wizard):
                    final_context.append(temp)
                    
            else:
                if i%2==0 and len(final_context)<len(current_wizard):
                    final_context.append(temp)
        
        context.append(final_context)
    
    return context, knowledge, wizard, dict2cnt

print("Getting 22311 conversation's context, knowledge, wizard utterances..")
t1=time.time()
context, knowledge, wizard, dict2cnt=load_data(json_data)        
t2=time.time()
print("This took: "+str(t2-t1)+" seconds")

Getting 22311 conversation's context, knowledge, wizard utterances..
This took: 11.647503852844238 seconds


In [10]:
def after_load_process(context,knowledge,wizard):
    input_context=[]
    input_knowledge=[]
    output_wizard=[]
    for i in range(len(wizard)):
        for c in context[i]:
            input_context.append(c)
        for k in knowledge[i]:
            input_knowledge.append(k)
        for w in wizard[i]:
            output_wizard.append(w)

    input_knowledge2=[]
    index=0
    for i in range(50246):
        r=dict2cnt[i]
        tmp=[]
        fr=index
        to=index+r
        for j in range(fr,to):
            tmp.append(input_knowledge[index])
            index=index+1
        input_knowledge2.append(tmp)
    input_knowledge3=[]
    for i in range(len(input_knowledge2)):
        g=[]
        for j in range(len(input_knowledge2[i])):
            s=""
            for k in range(len(input_knowledge2[i][j])):
                s=s+" "+input_knowledge2[i][j][k]
            s=s.strip()
            g.append(s)
        input_knowledge3.append(g)
    return input_context,input_knowledge3,output_wizard

print("Converting the extracted sentences into lists...")
t1=time.time()
final_context, final_knowledge, final_wizard=after_load_process(context,knowledge,wizard)
t2=time.time()
print("This took: "+str(t2-t1)+" seconds")

Converting the extracted sentences into lists...
This took: 1.40059232711792 seconds


In [11]:
##############################Sentence Length Analysis#############################

avg_wizard=0.0
no_wizard=0
no_knowledge=0
no_context=0
for i in range(len(final_wizard)):
    avg_wizard+=len(final_wizard[i].split())
    no_wizard+=1
    
avg_context=0.0
for i in range(len(final_context)):
    for j in range(len(final_context[i])):
        avg_context+=len(final_context[i][j].split())
        no_context+=1
        
avg_knowledge=0.0

for i in range(len(final_knowledge)):
    for j in range(len(final_knowledge[i])):
        avg_knowledge+=len(final_knowledge[i][j].split())
        no_knowledge+=1
        
avg_sentence=(avg_knowledge+avg_wizard+avg_context)/(no_wizard+no_context+no_knowledge)
print("Average Sentence Length: "+str(avg_sentence))
avg_wizard=avg_wizard/len(final_wizard)
avg_context=avg_context/no_context
avg_knowledge=avg_knowledge/no_knowledge
print("Average wizard's sentence length: "+str(avg_wizard))
print("Average knowledge's sentence length: "+str(avg_knowledge))
print("Average context's sentence length: "+str(avg_context))

Average Sentence Length: 69.08618384119995
Average wizard's sentence length: 18.304860088365242
Average knowledge's sentence length: 113.25242616631711
Average context's sentence length: 12.696140032022155


In [0]:

max_length=15

def process(sentence):
    """
        Preprocessing sentencing to make them of equal length and appending and terminating them with
        start (<START>) and end (<END>) token.
        
    """
    
    
    words=sentence.split()
    sentence_length=max_length-2
    if len(words)<=sentence_length:
        for i in range(sentence_length-len(words)):
            words.append("PAD")
    else:
        words=words[:sentence_length]
    
    res=""
    words.append("<END>")
    words.insert(0,"<START>")
    assert len(words)==max_length
    for w in words:
        res=res+" "+w
    res=res.strip()
    return res

In [13]:
vocab=vocabulary()

def process_loaded(input_context,input_knowledge,output_wizard):
    """ Processing sentences and forming vocabulary
    """
    
    for i in range(len(output_wizard)):
        output_wizard[i]=process(output_wizard[i])
        vocab.addSentence(output_wizard[i])
        assert len(output_wizard[i].split())==max_length
    
    for i in range(len(input_knowledge)):
        for j in range(len(input_knowledge[i])):
            input_knowledge[i][j]=process(input_knowledge[i][j])
            vocab.addSentence(input_knowledge[i][j])
            assert len(input_knowledge[i][j].split())==max_length
        
    for i in range(len(input_context)):
        for j in range(len(input_context[i])):
            input_context[i][j]=process(input_context[i][j])
            vocab.addSentence(input_context[i][j])
            assert len(input_context[i][j].split())==max_length
    return input_context, input_knowledge, output_wizard

print("Processing fetched sentences...")
t1=time.time()
p_context,p_knowledge,p_wizard=process_loaded(final_context,final_knowledge,final_wizard)
t2=time.time()
print("This process took: "+str(t2-t1))

Processing fetched sentences...
This process took: 9.385824918746948


In [14]:
##########################SANITY CHECK##########################

assert len(p_wizard)==50246
assert len(p_context)==50246
assert len(p_knowledge)==50246

print("Running Sanity Checks on the data...")

t1=time.time()

for i in range(len(p_wizard)):
    assert len(p_wizard[i].split())==max_length
    
for i in range(len(p_context)):
    for j in range(len(p_context[i])):
        assert len(p_context[i][j].split())==max_length
        
for i in range(len(p_knowledge)):
    for j in range(len(p_knowledge[i])):
        assert len(p_knowledge[i][j].split())==max_length

t2=time.time()
print("Everything seems fine.")        
print("Sanity Checks took: "+str(t2-t1)+" seconds")
 
################################################################

Running Sanity Checks on the data...
Everything seems fine.
Sanity Checks took: 0.693000078201294 seconds


In [19]:
print(len(p_wizard[0].split(" ")))
print((p_context[1]))

15
['<START> science fiction PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD <END>', '<START> i think science fiction is an amazing genre for anything future science technology <END>', '<START> im a huge fan of science fiction myself PAD PAD PAD PAD PAD <END>']
